In [42]:
# RGB vs LAB
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import skimage.morphology as smo
from skimage.io import imread
import itertools
from skimage import io, color

from sklearn.model_selection import train_test_split

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [43]:
def build_model(image_size, channels, start_neurons, dense_neurons, classes, model_name = "model_1"):
     
    input_tensor = tf.keras.layers.Input(shape = [image_size, image_size, channels])

    conv_1 = tf.keras.layers.Conv2D(filters = start_neurons * 1, 
                                    kernel_size = (3, 3),
                                    strides = (1, 1), 
                                    activation = tf.keras.activations.relu,
                                    padding = "same")(input_tensor)
    conv_2 = tf.keras.layers.Conv2D(filters = start_neurons * 1, 
                                    kernel_size = (3, 3),
                                    strides = (1, 1), 
                                    activation = tf.keras.activations.relu,
                                    padding = "same")(conv_1)
    conv_3 = tf.keras.layers.Conv2D(filters = start_neurons * 1, 
                                    kernel_size = (3, 3),
                                    strides = (1, 1), 
                                    activation = tf.keras.activations.relu,
                                    padding = "same")(conv_2)
    pool_1 = tf.keras.layers.MaxPool2D(pool_size = (2, 2),
                                       strides = (2, 2))(conv_3)

    flatten = tf.keras.layers.Flatten()(pool_1)
    dense = tf.keras.layers.Dense(units = dense_neurons, activation = tf.keras.activations.relu)(flatten)
    output_tensor = tf.keras.layers.Dense(units = classes, activation = tf.keras.activations.softmax)(dense)

    model = tf.keras.models.Model(inputs = input_tensor, 
                                  outputs = output_tensor, 
                                  name = model_name)
    
    return model

In [44]:
def expandgrid(*itrs):
    product = list(itertools.product(*itrs))
    return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}

In [48]:
def cifar_10_RGB(results_directory_ = "Results_7",
                 model_name_ = "model",
                 start_neurons_ = 16,
                 dense_neurons_ = 256,
                 epochs_ = 10,
                 batch_size_ = 1024,
                 augmentation_ = True):
    
    results_directory = os.path.join("D:/GitHub/PhD_Repository", results_directory_)

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    print("Split: train, validation and test")

    np.random.seed(42)
    tf.random.set_seed(42)
    X = np.arange(x_train.shape[0])
    y = np.arange(x_train.shape[0])

    classes = len(np.unique(y_train))

    x_train_id, x_validation_id, y_train_id, y_validation_id = train_test_split(X, y, test_size = 0.2, random_state = 42)

    x_train_ = x_train[x_train_id, :, :, :]
    y_train_ = y_train[y_train_id]

    x_valid_ = x_train[x_validation_id, :, :, :]
    y_valid_ = y_train[y_validation_id]

    x_test_ = x_test
    y_test_ = y_test

    y_train_ = tf.keras.utils.to_categorical(y_train_, classes)
    y_valid_ = tf.keras.utils.to_categorical(y_valid_, classes)
    y_test_ = tf.keras.utils.to_categorical(y_test_, classes)

    print(x_train_.shape)
    print(y_train_.shape)
    print(x_valid_.shape)
    print(y_valid_.shape)
    print(x_test_.shape)
    print(y_test_.shape)
    
    image_size = x_train_.shape[1]
    channels = x_train_.shape[3]
    early_stopping = int(epochs_ * 0.1)

    print("Build data generators")
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range = 90,
            width_shift_range = np.ceil(0.1 * image_size), 
            height_shift_range = np.ceil(0.1 * image_size),
            horizontal_flip = True, 
            vertical_flip = True,
            fill_mode = 'nearest')

    test_validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range = 0,
            width_shift_range = 0, 
            height_shift_range = 0,
            horizontal_flip = False, 
            vertical_flip = False,
            fill_mode = 'nearest')

    callbacks = [tf.keras.callbacks.EarlyStopping(patience = early_stopping, monitor = 'val_accuracy', verbose = 2),
                 tf.keras.callbacks.ModelCheckpoint(filepath = os.path.join(results_directory, "weights.h5"),
                                                    save_weights_only=True,
                                                    monitor='val_accuracy',
                                                    mode='max',
                                                    save_best_only=True,
                                                    verbose = 2)]

    model = build_model(image_size = image_size,
                        channels = channels, 
                        start_neurons = start_neurons_,
                        dense_neurons = dense_neurons_, 
                        classes = classes,
                        model_name = model_name_)
    print(model.summary())

    model.compile(optimizer = tf.keras.optimizers.Adam(),
                      loss = tf.keras.losses.categorical_crossentropy,
                      metrics = ["accuracy"])

    tf.keras.backend.clear_session()

    if augmentation_ == True:
        print("Augmentation mode on")
        train_generator = train_datagen.flow(x_train_, y_train_, batch_size = batch_size_)
    else:
        print("Augmentation mode off")
        train_generator = test_validation_datagen.flow(x_train_, y_train_, batch_size = batch_size_)

    validation_generator = test_validation_datagen.flow(x_valid_, y_valid_, batch_size = batch_size_)

    model_results = model.fit(train_generator,
                              validation_data = validation_generator,
                              steps_per_epoch = np.ceil(train_generator.n / batch_size_),
                              validation_steps = np.ceil(validation_generator.n / batch_size_),
                              epochs = epochs_,
                              shuffle = True,
                              callbacks = callbacks)

    model = build_model(image_size = image_size,
                        channels = channels, 
                        start_neurons = start_neurons_,
                        dense_neurons = dense_neurons_, 
                        classes = classes,
                        model_name = model_name_)

    model.load_weights(os.path.join(results_directory, "weights.h5"))

    model.compile(optimizer = tf.keras.optimizers.Adam(),
                  loss = tf.keras.losses.categorical_crossentropy,
                  metrics = ["accuracy"])

    model_results_pd = pd.DataFrame(model_results.history)
    model_results_pd["model"] = model_name_
    model_results_pd["epoch"] = np.array(model_results.epoch) + 1
    model_results_pd["start_neurons"] = start_neurons_
    model_results_pd["dense_neurons"] = dense_neurons_
    model_results_pd["epochs"] = epochs_
    model_results_pd["batch_size"] = batch_size_
    model_results_pd["augmentation"] = augmentation_
    model_results_pd.to_csv(os.path.join(results_directory,  model_name_ + "_history.csv"))

    model_train_accuracy = model.evaluate(x_train_, y_train_)[1]
    model_validation_accuracy = model.evaluate(x_valid_, y_valid_)[1]
    model_test_accuracy = model.evaluate(x_test_, y_test_)[1]

    print("Train accuracy:", model_train_accuracy)
    print("Validation accuracy:", model_validation_accuracy)
    print("Test accuracy:", model_test_accuracy)

    evaluation_results = pd.DataFrame({"Dataset" : ["train", "validation", "test"],
                                       "Accuracy" : [model_train_accuracy, model_validation_accuracy, model_test_accuracy],
                                       "Model_Name" : [model_name_] * 3})
    evaluation_results["epochs"] = model_results_pd.shape[0]
    evaluation_results["start_neurons"] = start_neurons_
    evaluation_results["dense_neurons"] = dense_neurons_
    evaluation_results["epochs"] = epochs_
    evaluation_results["batch_size"] = batch_size_
    evaluation_results["augmentation"] = augmentation_
    evaluation_results.to_csv(os.path.join(results_directory,  model_name_ + "_evaluation.csv"))

    del model
    del model_results_pd
    del evaluation_results
    os.remove(os.path.join(results_directory, "weights.h5"))

In [ ]:
def cifar_10_LAB(results_directory_ = "Results_7",
                 model_name_ = "model",
                 start_neurons_ = 16,
                 dense_neurons_ = 256,
                 epochs_ = 10,
                 batch_size_ = 1024,
                 augmentation_ = True):
    
    results_directory = os.path.join("D:/GitHub/PhD_Repository", results_directory_)

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    print("LAB transformation")
    x_train = [color.rgb2lab(i) for i in x_train]
    x_train = np.array(x_train)

    x_test = [color.rgb2lab(i) for i in x_test]
    x_test = np.array(x_test)
    
    print("Split: train, validation and test")

    np.random.seed(42)
    tf.random.set_seed(42)
    X = np.arange(x_train.shape[0])
    y = np.arange(x_train.shape[0])

    classes = len(np.unique(y_train))

    x_train_id, x_validation_id, y_train_id, y_validation_id = train_test_split(X, y, test_size = 0.2, random_state = 42)

    x_train_ = x_train[x_train_id, :, :, :]
    y_train_ = y_train[y_train_id]

    x_valid_ = x_train[x_validation_id, :, :, :]
    y_valid_ = y_train[y_validation_id]

    x_test_ = x_test
    y_test_ = y_test

    y_train_ = tf.keras.utils.to_categorical(y_train_, classes)
    y_valid_ = tf.keras.utils.to_categorical(y_valid_, classes)
    y_test_ = tf.keras.utils.to_categorical(y_test_, classes)

    print(x_train_.shape)
    print(y_train_.shape)
    print(x_valid_.shape)
    print(y_valid_.shape)
    print(x_test_.shape)
    print(y_test_.shape)
    
    image_size = x_train_.shape[1]
    channels = x_train_.shape[3]
    early_stopping = int(epochs_ * 0.1)

    print("Build data generators")
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range = 90,
            width_shift_range = np.ceil(0.1 * image_size), 
            height_shift_range = np.ceil(0.1 * image_size),
            horizontal_flip = True, 
            vertical_flip = True,
            fill_mode = 'nearest')

    test_validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range = 0,
            width_shift_range = 0, 
            height_shift_range = 0,
            horizontal_flip = False, 
            vertical_flip = False,
            fill_mode = 'nearest')

    callbacks = [tf.keras.callbacks.EarlyStopping(patience = early_stopping, monitor = 'val_accuracy', verbose = 2),
                 tf.keras.callbacks.ModelCheckpoint(filepath = os.path.join(results_directory, "weights.h5"),
                                                    save_weights_only=True,
                                                    monitor='val_accuracy',
                                                    mode='max',
                                                    save_best_only=True,
                                                    verbose = 2)]

    model = build_model(image_size = image_size,
                        channels = channels, 
                        start_neurons = start_neurons_,
                        dense_neurons = dense_neurons_, 
                        classes = classes,
                        model_name = model_name_)
    print(model.summary())

    model.compile(optimizer = tf.keras.optimizers.Adam(),
                      loss = tf.keras.losses.categorical_crossentropy,
                      metrics = ["accuracy"])

    tf.keras.backend.clear_session()

    if augmentation_ == True:
        print("Augmentation mode on")
        train_generator = train_datagen.flow(x_train_, y_train_, batch_size = batch_size_)
    else:
        print("Augmentation mode off")
        train_generator = test_validation_datagen.flow(x_train_, y_train_, batch_size = batch_size_)

    validation_generator = test_validation_datagen.flow(x_valid_, y_valid_, batch_size = batch_size_)

    model_results = model.fit(train_generator,
                              validation_data = validation_generator,
                              steps_per_epoch = np.ceil(train_generator.n / batch_size_),
                              validation_steps = np.ceil(validation_generator.n / batch_size_),
                              epochs = epochs_,
                              shuffle = True,
                              callbacks = callbacks)

    model = build_model(image_size = image_size,
                        channels = channels, 
                        start_neurons = start_neurons_,
                        dense_neurons = dense_neurons_, 
                        classes = classes,
                        model_name = model_name_)

    model.load_weights(os.path.join(results_directory, "weights.h5"))

    model.compile(optimizer = tf.keras.optimizers.Adam(),
                  loss = tf.keras.losses.categorical_crossentropy,
                  metrics = ["accuracy"])

    model_results_pd = pd.DataFrame(model_results.history)
    model_results_pd["model"] = model_name_
    model_results_pd["epoch"] = np.array(model_results.epoch) + 1
    model_results_pd["start_neurons"] = start_neurons_
    model_results_pd["dense_neurons"] = dense_neurons_
    model_results_pd["epochs"] = epochs_
    model_results_pd["batch_size"] = batch_size_
    model_results_pd["augmentation"] = augmentation_
    model_results_pd.to_csv(os.path.join(results_directory,  model_name_ + "_history.csv"))

    model_train_accuracy = model.evaluate(x_train_, y_train_)[1]
    model_validation_accuracy = model.evaluate(x_valid_, y_valid_)[1]
    model_test_accuracy = model.evaluate(x_test_, y_test_)[1]

    print("Train accuracy:", model_train_accuracy)
    print("Validation accuracy:", model_validation_accuracy)
    print("Test accuracy:", model_test_accuracy)

    evaluation_results = pd.DataFrame({"Dataset" : ["train", "validation", "test"],
                                       "Accuracy" : [model_train_accuracy, model_validation_accuracy, model_test_accuracy],
                                       "Model_Name" : [model_name_] * 3})
    evaluation_results["epochs"] = model_results_pd.shape[0]
    evaluation_results["start_neurons"] = start_neurons_
    evaluation_results["dense_neurons"] = dense_neurons_
    evaluation_results["epochs"] = epochs_
    evaluation_results["batch_size"] = batch_size_
    evaluation_results["augmentation"] = augmentation_
    evaluation_results.to_csv(os.path.join(results_directory,  model_name_ + "_evaluation.csv"))

    del model
    del model_results_pd
    del evaluation_results
    os.remove(os.path.join(results_directory, "weights.h5"))

In [49]:
start_neurons_hp = [16, 32, 64]
dense_neurons_hp = [128, 256, 512]

grid = pd.DataFrame(expandgrid(start_neurons_hp, dense_neurons_hp))
grid["id"] = list(np.arange(grid.shape[0]))
grid.columns = ["start_neurons_hp", "dense_neurons_hp", "id"]
grid

,start_neurons_hp,dense_neurons_hp,id
0,16,128,0
1,16,256,1
2,16,512,2
3,32,128,3
4,32,256,4
5,32,512,5
6,64,128,6
7,64,256,7
8,64,512,8


In [50]:
for i in np.arange(grid.shape[0]):
    model_name_ = "RGB" + str(i)
    print(model_name_)
    
    cifar_10_RGB(results_directory_ = "Results_7",
                 model_name_ = model_name_,
                 start_neurons_ = grid["start_neurons_hp"][i],
                 dense_neurons_ = grid["dense_neurons_hp"][i],
                 epochs_ = 100,
                 batch_size_ = 512,
                 augmentation_ = True)

RGB0
Split: train, validation and test
(40000, 32, 32, 3)
(40000, 10)
(10000, 32, 32, 3)
(10000, 10)
(10000, 32, 32, 3)
(10000, 10)
Build data generators
Model: "RGB0"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4096

In [ ]:
for i in np.arange(grid.shape[0]):
    model_name_ = "LAB" + str(i)
    print(model_name_)
    
    cifar_10_RGB(results_directory_ = "Results_7",
                 model_name_ = model_name_,
                 start_neurons_ = grid["start_neurons_hp"][i],
                 dense_neurons_ = grid["dense_neurons_hp"][i],
                 epochs_ = 100,
                 batch_size_ = 512,
                 augmentation_ = True)

LAB0
Split: train, validation and test
(40000, 32, 32, 3)
(40000, 10)
(10000, 32, 32, 3)
(10000, 10)
(10000, 32, 32, 3)
(10000, 10)
Build data generators
Model: "LAB0"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096